In [1]:
from pathlib import Path
from shamo.model import FEModel

model_name = "example_model"
model_path = Path(
    "./derivatives/{name}/{name}.json".format(name=model_name))
model = FEModel.load(model_path)

In [2]:
from shamo.problem.forward.eeg import EEGForwardProblem

problem = EEGForwardProblem()

problem.add_tissue("a", 1.0)
problem.add_tissue("b", 0.5)
problem.add_tissue("c", 0.25)

problem.set_reference("A")
problem.add_markers(["C", "G"])

problem.add_regions_of_interest(["b", "c"])

problem.set_undersampling_distance(50)

{'problem_type': 'eeg',
 'tissue_properties': {'a': {'sigma': 1.0, 'use_anisotropy': False},
  'b': {'sigma': 0.5, 'use_anisotropy': False},
  'c': {'sigma': 0.25, 'use_anisotropy': False}},
 'undersampling_distance': 50,
 'markers': ['C', 'G'],
 'reference': 'A',
 'region_of_interest': ['b', 'c']}

In [3]:
model.tissues

{'a': {'volume_group': 1,
  'volume_entity': [1],
  'surface_group': 2,
  'surface_entity': [1]},
 'b': {'volume_group': 3,
  'volume_entity': [2],
  'surface_group': 4,
  'surface_entity': [2]},
 'c': {'volume_group': 5,
  'volume_entity': [3],
  'surface_group': 6,
  'surface_entity': [3]}}

In [4]:
problem.solve(model, "example_problem", "./derivatives")

1 [2659]
